In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import collections
import machine_learning_helper as machine_learning_helper

In [2]:
df_train_users = pd.read_csv("cleaned_train_user.csv")
df_test_users = pd.read_csv("cleaned_test_user.csv")
df_time_mean_user_id = pd.read_csv("time_mean_user_id.csv")
df_time_total_user_id = pd.read_csv("time_total_user_id.csv")
df_total_action_user_id = pd.read_csv("total_action_user_id.csv")

In [3]:
df_total_action_user_id.columns = ['id','action']
df_sessions = pd.merge(df_time_mean_user_id, df_time_total_user_id, on='id', how='outer')
df_sessions = pd.merge(df_sessions, df_total_action_user_id, on='id', how='outer')
df_sessions.columns = ['id','time_mean_user','time_total_user','action']
df_sessions.head()

print("X_train has dimension:",df_train_users.shape)
print("X_test has dimension:",df_test_users.shape)

X_train has dimension: (213451, 16)
X_test has dimension: (62096, 15)


In [4]:
y_labels, label_enc = machine_learning_helper.buildTargetMat(df_train_users)

In [5]:
df_train_len = df_train_users.shape[0]
df_train = df_train_users.drop(['country_destination'],axis=1)
df_all = pd.concat((df_train_users, df_test_users), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_sessions, on='id', how='left', left_index=True)

print(df_all['action'].isnull().values.sum())
print(len(df_train['id'].unique()))
print(len(df_train['id']))

141490
213451
213451


In [6]:
X_train, X_test = machine_learning_helper.buildFeatsMat(df_train_users, df_test_users, df_sessions)

In [7]:
print("X_train has dimension:",X_train.shape)
print("X_test has dimension:",X_test.shape)

X_train has dimension: (213451, 156)
X_test has dimension: (62096, 156)


In [29]:
train_data = X_train#.values.T.tolist()
# Define computational graph (CG)
batch_size = train_data.shape[0]         # batch size
d = train_data.shape[1]  # data dimensionality
nc =  10   # number of classes

# CG inputs
xin = tf.placeholder(tf.float32,[batch_size,d]); #print('xin=',xin,xin.get_shape())
y_label = tf.placeholder(tf.float32,[batch_size,nc]); #print('y_label=',y_label,y_label.get_shape())

# Fully Connected layer
W = tf.Variable(tf.truncated_normal([d,nc], stddev=tf.sqrt(6./tf.to_float(d+nc)) )); #print('W=',W.get_shape())
b = tf.Variable(tf.zeros([nc])); #print('b=',b.get_shape())
y = tf.matmul(xin, W); #print('y1=',y,y.get_shape())
y += b; #print('y2=',y,y.get_shape())

# Softmax
y = tf.nn.softmax(y); #print('y3=',y,y.get_shape())

# Loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(y), 1))
total_loss = cross_entropy

# Optimization scheme
#train_step = tf.train.GradientDescentOptimizer(0.025).minimize(total_loss)
train_step = tf.train.AdamOptimizer(0.001).minimize(total_loss)

In [30]:
train_labels = y_labels
test_data = X_test

# Run Computational Graph
n = train_data.shape[0]
indices = collections.deque()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Run CG for variable training
y,_,total_loss_o = sess.run([y,train_step,total_loss], feed_dict={xin: X_train, y_label: y_labels})

# Run CG for test set
y,_,_ = sess.run(feed_dict={xin: X_test, y_label: y_labels})
        

ValueError: Cannot feed value of shape (213451,) for Tensor 'Placeholder_9:0', which has shape '(213451, 10)'

In [18]:
train_data.shape[0] 

213451

In [19]:
train_labels = y_labels
test_data = X_test

# Run Computational Graph
n = train_data.shape[0]
indices = collections.deque()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(10001):
    
    # Batch extraction
    if len(indices) < batch_size:
        indices.extend(np.random.permutation(n)) 
    idx = [indices.popleft() for i in range(batch_size)]
    batch_x, batch_y = train_data.loc[idx,:], train_labels[idx] 
    print(batch_x.shape,batch_y.shape)
    
    # Run CG for variable training
    _,acc_train,total_loss_o = sess.run([train_step,total_loss], feed_dict={xin: X_train, y_label: y_labels})
    
    # Run CG for test set
    if not i%1000:
        print('\nIteration i=',i,', loss=',total_loss_o)
        acc_test = sess.run(feed_dict={xin: X_test, y_label: y_labels})
        

(354271, 156) (213451,)


ValueError: Cannot feed value of shape (213451,) for Tensor 'Placeholder_7:0', which has shape '(213451, 213451)'

In [ ]:
a = X_train.loc[range(5),:]
a

In [28]:
y_labels

array([ 7,  7, 10, ...,  7,  7,  7])

LabelEncoder()